In [6]:
import nltk
import random
from gensim.models import Word2Vec
import numpy as np

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [7]:
random.seed(30) #for reproducibility!

In [12]:
# read one .tsv file
def read_tsv(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
    return lines

#1. 
filename = 'en-hr.news+reviews.profs.id+en+hr.tsv'
#2. 
#filename = 'en-ru.news+reviews.profs.id+en+ru.tsv'

tsv_lines = read_tsv(filename)

FileNotFoundError: [Errno 2] No such file or directory: 'en-hr.news+reviews.profs.id+en+hr.tsv'

In [9]:
print(len(tsv_lines))
print(tsv_lines[0])

# split the lines into a list of lists
tabs = [line.split('\t') for line in tsv_lines]
print(tabs[0])

NameError: name 'tsv_lines' is not defined

In [10]:
# separate english and finnish lines
en_lines = [line[2] for line in tabs]
fi_lines = [line[-1] for line in tabs]

len(en_lines), len(fi_lines)

NameError: name 'tabs' is not defined

In [11]:
# super cool. Now let's do it the gensim way.

In [955]:
#I create a single "sentence" out of the original and the translation

# import the stemmer
from nltk.stem.snowball import SnowballStemmer
stemmer_en = SnowballStemmer("english")
stemmer_other = SnowballStemmer("croatian")


aligned = []
for i in range(len(en_lines)):
    english = en_lines[i]
    other = fi_lines[i]
    english = nltk.wordpunct_tokenize(english.lower())
    other = nltk.wordpunct_tokenize(other.lower())

    # 3.
    # stem the words
    english_lines = [stemmer_en.stem(word) for word in english]
    other_lines = [stemmer_other.stem(word) for word in other]

    # 3.5 does it get better if we remove stopwords?
    from nltk.corpus import stopwords
    en_stop_words = set(stopwords.words('english'))
    english_lines = [word for word in english_lines if word not in en_stop_words]
    other_stop_words = set(stopwords.words('finnish'))
    other_lines = [word for word in other_lines if word not in other_stop_words]

    # 4.
    # add prefixes
    english_lines = ['en_' + word for word in english_lines]
    other_lines = ['fi_' + word for word in other_lines]

    multi = english_lines + other_lines
    # tokenize multi
    #multi = nltk.wordpunct_tokenize(multi.lower())
    
    random.shuffle(multi) #<<< Different shufflings still return good performances
    #It seems necessary with this setting to shuffle words' contexts.
    #Concatenating the sentences makes the far away context too irrelevant and so the languages stay divided. 

    aligned.append(multi)
    
aligned[0] # now Finnish and English are shuffled!

ValueError: The language 'croatian' is not supported.

In [931]:
aligned[10]

['fi_,',
 'fi_suosittel',
 'fi_.',
 'en_.',
 'en_recommend',
 'en_tri',
 'fi_kokeilet',
 'en_guy']

In [932]:
average_length = sum([len(s) for s in aligned])/len(aligned)
print(average_length)
std_length = np.std([len(s) for s in aligned])
print(std_length)

18.7192118226601
13.433176170916354


In [933]:
help(Word2Vec)

Help on class Word2Vec in module gensim.models.word2vec:

class Word2Vec(gensim.utils.SaveLoad)
 |  Word2Vec(sentences=None, corpus_file=None, vector_size=100, alpha=0.025, window=5, min_count=5, max_vocab_size=None, sample=0.001, seed=1, workers=3, min_alpha=0.0001, sg=0, hs=0, negative=5, ns_exponent=0.75, cbow_mean=1, hashfxn=<built-in function hash>, epochs=5, null_word=0, trim_rule=None, sorted_vocab=1, batch_words=10000, compute_loss=False, callbacks=(), comment=None, max_final_vocab=None, shrink_windows=True)
 |  
 |  Method resolution order:
 |      Word2Vec
 |      gensim.utils.SaveLoad
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, sentences=None, corpus_file=None, vector_size=100, alpha=0.025, window=5, min_count=5, max_vocab_size=None, sample=0.001, seed=1, workers=3, min_alpha=0.0001, sg=0, hs=0, negative=5, ns_exponent=0.75, cbow_mean=1, hashfxn=<built-in function hash>, epochs=5, null_word=0, trim_rule=None, sorted_vocab=1, batch_words=100

In [934]:
%%time

#I train a model on the average + std deviation bilingual context of each word.
# 300 is the standard w2v size
window_size = int(average_length + std_length)
madmod0 = Word2Vec(aligned,window=window_size, vector_size=200, min_count=0, workers=4, epochs=25)

CPU times: user 892 ms, sys: 18.4 ms, total: 911 ms
Wall time: 519 ms


In [935]:
print(madmod0.wv.index_to_key[:100])

['fi_.', 'en_.', 'fi_,', 'en_,', "en_'", 'fi_-', 'fi_mut', 'fi_!', 'en_!', 'en_one', 'en_-', 'fi_kirj', 'en_book', 'fi_hyvä', 'fi_ova', 'en_like', 'fi_minu', 'en_use', 'fi_:', 'fi_vain', 'en_veri', 'en_great', 'en_product', 'fi_tode', 'en_work', 'fi_ost', 'en_good', 'fi_hyv', 'en_time', 'en_onli', 'fi_ol', 'en_game', 'en_would', 'fi_kaik', 'en_get', 'en_love', 'en_"', 'fi_...', 'fi_kosk', 'fi_sil', 'fi_palj', 'fi_tuot', 'fi_tuote', 'fi_sama', 'en_play', 'en_even', 'en_buy', 'en_make', 'en_look', 'en_year', 'en_back', 'en_;', 'en_much', 'en_becaus', 'fi_;', 'fi_kan', 'fi_34', 'en_34', 'en_tri', 'en_bought', 'en_well', 'fi_myös', 'en_...', 'fi_rakast', 'fi_(', 'en_(', 'en_need', 'en_doe', 'en_recommend', 'fi_toimi', 'en_realli', 'fi_letku', 'fi_&#', 'en_&#', 'fi_voi', 'en_cake', 'en_hose', 'en_first', 'en_want', 'en_ani', 'fi_olu', 'fi_versio', 'en_find', 'en_money', 'en_mac', 'fi_mac', 'fi_jot', 'en_littl', 'en_/', 'en_know', 'fi_pel', 'en_see', 'en_old', 'en_5', 'fi_tarvits', 'fi_os', 

In [936]:
# count the number of times you have given words in the aligned sentences
count = 0
for s in aligned:
    if 'book' in s:
        count += 1

print(count)

0


In [937]:
# strange training.

In [938]:
# word list
FINNISH
kirj - book 
tarjous - offer

NameError: name 'FINNISH' is not defined

In [939]:
# let's check the model

madmod0.wv.most_similar('en_book') 

[('fi_kirj', 0.9980058073997498),
 ('en_style', 0.9964943528175354),
 ('en_essenti', 0.9946607351303101),
 ('fi_of', 0.9941142201423645),
 ('en_along', 0.9938984513282776),
 ('en_theori', 0.9935122728347778),
 ('fi_sivu', 0.9933909177780151),
 ('en_page', 0.9930604696273804),
 ('en_text', 0.9930580854415894),
 ('en_decor', 0.9918487071990967)]

In [940]:
madmod0.wv.most_similar('kirj')

KeyError: "Key 'kirj' not present in vocabulary"

In [941]:
# check the most frequent words



In [942]:
madmod0.wv.most_similar('kun', topn=3), madmod0.wv.most_similar('when', topn=3)

KeyError: "Key 'kun' not present in vocabulary"

In [943]:
madmod0.wv.most_similar('en_today') # kirjia words

[('fi_avu', 0.9987568855285645),
 ('en_light', 0.9986424446105957),
 ('fi_tule', 0.9986009001731873),
 ('en_magnifi', 0.9985554218292236),
 ('en_idea', 0.9985470771789551),
 ('fi_saapui', 0.9985215663909912),
 ('en_order', 0.998475193977356),
 ('en_howev', 0.9984341263771057),
 ('en_might', 0.9984232783317566),
 ('en_use', 0.9984159469604492)]

In [944]:
# these languages are too inflectional for this to work on few sentences. Or the translations are strange. We need something else. 

In [945]:
# let's try the oldest way. A word to document matrix.

all_words = set()
for s in aligned:
    all_words.update(s)
    

# For each word, note whether and how often it happens in each aligned sentence.

matrix = {}
for w in all_words:
    #matrix[w] = [s.count(w) for s in aligned]
    matrix[w] = [1 if w in s else 0 for s in aligned]

# let's check the matrix




In [946]:
# now use some package to check the cosine similarity between the vectors of each word

from sklearn.metrics.pairwise import cosine_similarity

# let's check the similarity between the vectors of 'book' and 'kirja' using the matrix

cosine_similarity([matrix['en_book']], [matrix['fi_kirj']])

array([[0.90099965]])

In [947]:
# what is the closest word to 'book' in the matrix? (use the matrix)

word = 'en_love'

# stem the word
#word = stemmer_en.stem(word)
#print(word)

to_sort = []
for w in all_words:
    if w != word:
        to_sort.append((cosine_similarity([matrix[word]], [matrix[w]]),w))

In [948]:
# find the closest n words to the word
n=10
sorted(to_sort, reverse=True)[:n] # geh. attack is a good example. 

[(array([[0.84084099]]), 'fi_rakast'),
 (array([[0.24253563]]), 'fi_kotelo'),
 (array([[0.21004201]]), 'fi_vanh'),
 (array([[0.19802951]]), 'fi_pian'),
 (array([[0.19802951]]), 'fi_macely'),
 (array([[0.19802951]]), 'en_mace'),
 (array([[0.17712298]]), 'en_case'),
 (array([[0.17149859]]), 'fi_äärimmäis'),
 (array([[0.17149859]]), 'fi_äänit'),
 (array([[0.17149859]]), 'fi_äid')]

In [923]:
# can i go back from stemmed term to original term?


In [949]:
madmod0.wv.most_similar(word) # kirjia words

[('fi_pelaam', 0.9955849647521973),
 ('fi_pel', 0.9951846599578857),
 ('en_math', 0.9945418238639832),
 ('fi_ehdottom', 0.9942481517791748),
 ('en_seven', 0.9941017031669617),
 ('en_fun', 0.9939164519309998),
 ('en_expect', 0.9939135909080505),
 ('fi_tark', 0.9937585592269897),
 ('fi_tode', 0.993554949760437),
 ('fi_hausk', 0.993508517742157)]

In [ ]:
# Ok now.
# 1. Save the matrices. 
# 2. Make sure one can retrieve and use them.
# 3. Make two pages of detailings. 

In [950]:
# how do I save this matrix?
# I can use a json file.
import json
with open('term2doc_nostopwords_en-fi.reviews.studs.id+en+fi.json', 'w') as fp:
    json.dump(matrix, fp)
